<a href="https://colab.research.google.com/github/buikeith112/customer-support/blob/main/Headstarter_RAG_Workshop_Completed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter RAG Workshop

**Skills: OpenAI, LangChain, Pinecone**


### Workshop Recording: https://www.loom.com/share/75af4269ab66450e943160c199895aa7


**Other Resources:**
- [Get your OpenAI API Key](https://platform.openai.com/settings/profile?tab=api-keys)
- [Get your Pinecone API Key](https://www.pinecone.io/)
- [Get your OpenRouter API Key](https://openrouter.ai/settings/keys)
- [JavaScript Code for RAG](https://js.langchain.com/v0.2/docs/tutorials/rag)
- [RAG with an in-memory database in Next.js](https://sdk.vercel.ai/examples/node/generating-text/rag)


### What is RAG anyway?


Retrieval-Augmented Generation (RAG) is a technique primarily used in GenAI applications to improve the quality and accuracy of generated text by LLMs by combining two key processes: retrieval and generation.

### Breaking It Down:
#### Retrieval:

- Before generating a response, the system first looks up relevant information from a large database or knowledge base. This is like searching through a library or the internet to find the most useful facts, articles, or data related to the question or topic.

#### Generation:

- Once the relevant information is retrieved, the system then uses it to help generate a response. This is where the model, like GPT, creates new text (answers, explanations, etc.) based on the retrieved information.

#### Install relevant libraries

In [2]:
! pip install langchain langchain-community openai tiktoken pinecone-client langchain_pinecone unstructured pdfminer==20191125 pdfminer.six==20221105 pillow_heif unstructured_inference youtube-transcript-api pytube sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 33.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 997.5 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_

In [3]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, WebBaseLoader, YoutubeLoader, DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
from openai import OpenAI
import numpy as np
import tiktoken
import os

pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

#openai_api_key = userdata.get("OPENAI_API_KEY")
#os.environ['OPENAI_API_KEY'] = openai_api_key

# Initialize the OpenAI client

In [19]:
embeddings = OpenAIEmbeddings()
embed_model = "text-embedding-3-small"
openai_client = OpenAI()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

# Use HuggingFace & OpenRouter if you don't have an OpenAI account with credits



In [5]:
# HuggingFace Embeddings
# Use this instead of OpenAI embeddings if you don't have an OpenAI account with credits

text = "This is a test document."

hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
query_result = hf_embeddings.embed_query(text)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
query_result

[-0.038338541984558105,
 0.12346471846103668,
 -0.02864297851920128,
 0.05365270376205444,
 0.008845366537570953,
 -0.03983934596180916,
 -0.07300589233636856,
 0.04777132719755173,
 -0.030462471768260002,
 0.05497974902391434,
 0.08505292981863022,
 0.03665666654706001,
 -0.005319973453879356,
 -0.002233141800388694,
 -0.06071099638938904,
 -0.027237920090556145,
 -0.01135166734457016,
 -0.042437683790922165,
 0.00912993960082531,
 0.10081552714109421,
 0.07578728348016739,
 0.06911715865135193,
 0.009857431054115295,
 -0.0018377641681581736,
 0.02624903991818428,
 0.03290243074297905,
 -0.07177437096834183,
 0.028384247794747353,
 0.06170954555273056,
 -0.052529532462358475,
 0.033661652356386185,
 0.07446812838315964,
 0.07536034286022186,
 0.03538404777646065,
 0.06713404506444931,
 0.010798045434057713,
 0.08167017996311188,
 0.016562897711992264,
 0.03283063694834709,
 0.036325663328170776,
 0.0021727988496422768,
 -0.09895738214254379,
 0.0050467848777771,
 0.05089650675654411,


In [7]:
# Free Llama 3.1 API via OpenRouter
# Use this instead of OpenAI if you don't have an OpenAI account with credits

openrouter_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=userdata.get("OPENROUTER_API_KEY")
)

## Initialize our text splitter
This is how we will chunk up the text to be retrieved during the RAG process

In [8]:
tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=100,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
)

# Understanding Embeddings

In [9]:
def get_embedding(text, model="all-MiniLM-L6-v2"):
    # Call the OpenAI API to get the embedding for the text
    response = hf_embeddings.embed_query(text)
    return response

def cosine_similarity_between_words(sentence1, sentence2):
    # Get embeddings for both words
    embedding1 = np.array(get_embedding(sentence1))
    embedding2 = np.array(get_embedding(sentence2))

    # Reshape embeddings for cosine_similarity function
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    print("Embedding for Sentence 1:", embedding1)
    print("\nEmbedding for Sentence 2:", embedding2)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity[0][0]


# Example usage
sentence1 = "I like walking to the park"
sentence2 = "I like walking to the office"


similarity = cosine_similarity_between_words(sentence1, sentence2)
print(f"\n\nCosine similarity between '{sentence1}' and '{sentence2}': {similarity:.4f}")


Embedding for Sentence 1: [[-7.94647262e-04 -4.52190749e-02  5.60034551e-02  4.00062464e-02
   7.82356039e-02 -3.10016028e-03  1.56902865e-01 -1.61643641e-03
   8.40177536e-02  7.29586333e-02 -2.27428153e-02 -1.00336559e-02
  -4.77766357e-02  5.78007065e-02  6.89263120e-02  2.29866221e-03
   3.41052189e-02  8.23902860e-02 -4.47453046e-03  1.18202856e-02
  -7.44135678e-02  2.10828464e-02  1.92200206e-02  5.48400655e-02
  -1.07110761e-01  8.79157037e-02 -1.64800771e-02  6.51672296e-03
  -6.67020795e-05 -4.27562976e-03 -8.20703059e-02  7.05852956e-02
  -1.80556532e-02  3.27348486e-02 -4.36549522e-02  9.93786659e-03
   5.78057803e-02 -6.92316219e-02  4.53142636e-02  4.96660285e-02
  -1.49475699e-02  5.79100735e-02  8.14058036e-02  2.62879906e-03
  -1.49136577e-02 -4.37886156e-02  2.26743110e-02 -3.19027528e-02
   1.00592583e-01  3.10835298e-02  1.30596399e-01  7.27660581e-03
   8.58721696e-03  7.95205031e-03 -7.91899022e-03  4.98277741e-03
  -8.22421089e-02  2.46388651e-02  5.11084683e-02 

# Load in a YouTube video and get its transcript

In [19]:
# Load in a YouTube video's transcript
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=rfscVS0vtbw", add_video_info=True)
data = loader.load()

print(data)

TypeError: YoutubeLoader.from_youtube_url() takes 2 positional arguments but 3 were given

In [11]:
texts = text_splitter.split_documents(data)

In [12]:
texts

[Document(metadata={'source': 'rfscVS0vtbw', 'title': 'Learn Python - Full Course for Beginners [Tutorial]', 'description': 'Unknown', 'view_count': 44702462, 'thumbnail_url': 'https://i.ytimg.com/vi/rfscVS0vtbw/hq720.jpg', 'publish_date': '2018-07-11 00:00:00', 'length': 16012, 'author': 'freeCodeCamp.org'}, page_content="In this course, I'm going to teach you everything you need to know to get started programming in Python. Now, Python is one of the most popular programming languages out there. And it's by far one of the most sought after for jobs. And so if you're trying to get a job or you're trying to, you know, automate your life, or you're trying to write awesome scripts to do a bunch of different things, then Python's for you. Honestly, more and more developers every day are moving their projects over to Python because it's such a powerful and it's such an easy to use language. A lot of programming languages out there just aren't very beginner friendly. There's a lot of syntax.

# Initialize Pinecone

In [16]:
vectorstore = PineconeVectorStore(index_name="customer-support", embedding=hf_embeddings)

index_name = "customer-support"

namespace = "python-youtube-videos"

# Insert data into Pinecone

Documentation: https://docs.pinecone.io/integrations/langchain#key-concepts

In [17]:
for document in texts:
    print("\n\n\n\n-------------------------")

    print(document.metadata, document.page_content)

    print('\n\n\n\n-------------------------')





-------------------------
{'source': 'rfscVS0vtbw', 'title': 'Learn Python - Full Course for Beginners [Tutorial]', 'description': 'Unknown', 'view_count': 44702462, 'thumbnail_url': 'https://i.ytimg.com/vi/rfscVS0vtbw/hq720.jpg', 'publish_date': '2018-07-11 00:00:00', 'length': 16012, 'author': 'freeCodeCamp.org'} In this course, I'm going to teach you everything you need to know to get started programming in Python. Now, Python is one of the most popular programming languages out there. And it's by far one of the most sought after for jobs. And so if you're trying to get a job or you're trying to, you know, automate your life, or you're trying to write awesome scripts to do a bunch of different things, then Python's for you. Honestly, more and more developers every day are moving their projects over to Python because it's such a powerful and it's such an easy to use language. A lot of programming languages out there just aren't very beginner friendly. There's a lot of syntax. The

In [18]:
vectorstore_from_texts = PineconeVectorStore.from_texts([f"Source: {t.metadata['source']}, Title: {t.metadata['title']} \n\nContent: {t.page_content}" for t in texts], embedding=hf_embeddings, index_name=index_name, namespace=namespace)

# Perform RAG

In [47]:
from pinecone import Pinecone

In [48]:
# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("customer-support")

In [49]:
query = "What does Aravind mention about pre-training and why it is important?"

In [66]:
raw_query_embedding = hf_embeddings.embed_query(
    query
    #model="text-embedding-3-small"
)

query_embedding = raw_query_embedding

In [54]:
query_embedding

[0.008593614213168621,
 0.012237180955708027,
 -0.05014544725418091,
 -0.030369197949767113,
 0.015887394547462463,
 -0.01117543876171112,
 0.014318762347102165,
 0.06232735142111778,
 0.00856869574636221,
 -0.031177805736660957,
 0.024387966841459274,
 0.010222614742815495,
 -0.05102444067597389,
 -0.00893334485590458,
 -0.009643557481467724,
 -0.05268789455294609,
 0.009386335499584675,
 0.026987655088305473,
 -0.01663076877593994,
 -0.009288323111832142,
 0.05328906700015068,
 -0.005031646229326725,
 -0.03086932748556137,
 -0.029297223314642906,
 -0.017517855390906334,
 0.0032198631670325994,
 -0.046020254492759705,
 0.0005555913667194545,
 -0.045086536556482315,
 -0.02356068044900894,
 -0.040326934307813644,
 0.025932393968105316,
 -0.041495371609926224,
 -0.001193337724544108,
 -0.016887405887246132,
 0.039472032338380814,
 0.04707902669906616,
 0.04126686975359917,
 -0.038189295679330826,
 0.02541879564523697,
 -0.012181906960904598,
 0.05078732222318649,
 -0.03346577286720276,
 

In [55]:
top_matches = pinecone_index.query(vector=query_embedding, top_k=10, include_metadata=True, namespace=namespace)

In [56]:
top_matches

{'matches': [{'id': '8c3e806e-e259-4170-9048-43f782a39e2c',
              'metadata': {'text': 'Source: e-gwvmhyU7A, Title: Aravind '
                                   'Srinivas: Perplexity CEO on Future of AI, '
                                   'Search & the Internet | Lex Fridman '
                                   'Podcast #434 \n'
                                   '\n'
                                   'Content: for each of those outputs, and '
                                   'you train the model on that. Now there are '
                                   "some prompts where it's not gonna get it "
                                   'right, now, instead of just\n'
                                   'training on the right answer, you ask it '
                                   'to produce an explanation: If you were '
                                   'given the right answer, what is the '
                                   'explanation you provided? You train on '
       

In [57]:
# Get the list of retrieved texts
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [58]:
contexts

['Source: e-gwvmhyU7A, Title: Aravind Srinivas: Perplexity CEO on Future of AI, Search & the Internet | Lex Fridman Podcast #434 \n\nContent: for each of those outputs, and you train the model on that. Now there are some prompts where it\'s not gonna get it right, now, instead of just\ntraining on the right answer, you ask it to produce an explanation: If you were given the right answer, what is the explanation you provided? You train on that. And for whatever you got, right, you just train on the whole string of prompt, explanation and output. This way, even if you didn\'t\narrive with the right answer, if you had been given the\nhint of the right answer, you\'re trying to, like, reason what would\'ve gotten me that right answer and then training on that. And mathematically you can prove that it\'s, like, related to the\nvariational lower bound with the latent. And I think it\'s a very interesting way to use natural language\nexplanations as a latent, that way you can refine the model

In [59]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [60]:
print(augmented_query)

<CONTEXT>
Source: e-gwvmhyU7A, Title: Aravind Srinivas: Perplexity CEO on Future of AI, Search & the Internet | Lex Fridman Podcast #434 

Content: for each of those outputs, and you train the model on that. Now there are some prompts where it's not gonna get it right, now, instead of just
training on the right answer, you ask it to produce an explanation: If you were given the right answer, what is the explanation you provided? You train on that. And for whatever you got, right, you just train on the whole string of prompt, explanation and output. This way, even if you didn't
arrive with the right answer, if you had been given the
hint of the right answer, you're trying to, like, reason what would've gotten me that right answer and then training on that. And mathematically you can prove that it's, like, related to the
variational lower bound with the latent. And I think it's a very interesting way to use natural language
explanations as a latent, that way you can refine the model itse

In [68]:
# Modify the prompt below as need to improve the response quality
primer = f"""You are a personal assistant. Answer any questions I have about the Youtube Video provided.
"""

res = openrouter_client.chat.completions.create(
    model="meta-llama/llama-3.1-8b-instruct:free",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)

openai_answer = res.choices[0].message.content

In [69]:
print(openai_answer)

Aravind mentions the concept of pre-training, which involves training a model on a broad range of tasks and data to improve its ability to reason and learn. He explains that this approach is important because it allows the model to develop a strong foundation in reasoning and problem-solving, which can then be applied to more specific tasks and domains.

Specifically, Aravind mentions that instead of just training the model on the correct output, they ask it to produce an explanation for the output, and then train the model on the entire string of prompt, explanation, and output. This approach, he says, is related to the variational lower bound with the latent, and it allows the model to refine its reasoning abilities and become a better reasoner.

Aravind also highlights the importance of pre-training in enabling the model to generalize to new tasks and domains, and to develop a more robust and transferable set of reasoning abilities. He suggests that this approach could be an importa

# Using OpenRouter

In [70]:
 # Check out different models here: https://openrouter.ai/docs/models

res = openrouter_client.chat.completions.create(
    model="mistralai/mistral-nemo",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)

answer = res.choices[0].message.content

In [71]:
print(answer)

Aravind mentions that pre-training is a crucial step in building large language models (LLMs) like Perplexity. It involves training the model on a vast amount of data to learn common sense, understanding of natural language, and various other skills. This step is essential because it enables the model to generate human-like text and understand the context of a conversation. After pre-training, the model can be fine-tuned on specific tasks or domains to improve its performance.


# Putting it all together

In [72]:
def perform_rag(query):
    raw_query_embedding = openai_client.embeddings.create(
        input=query,
        model="text-embedding-3-small"
    )

    query_embedding = raw_query_embedding.data[0].embedding

    top_matches = pinecone_index.query(vector=query_embedding, top_k=10, include_metadata=True, namespace=namespace)

    # Get the list of retrieved texts
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

    # Modify the prompt below as need to improve the response quality
    system_prompt = f"""You are an expert personal assistant. Answer any questions I have about the Youtube Video provided. You always answer questions based only on the context that you have been provided.
    """

    res = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    return res.choices[0].message.content


In [ ]:
perform_rag("What does Aravind mention about pre-training and why it is important?")

"Aravind Srinivas discusses the importance of pre-training in the development of effective AI models, highlighting its role in creating a foundation of general common sense that is crucial for the model's performance. Here are the key points he mentions about pre-training and its significance:\n\n1. **Foundational Stage**: Pre-training is the stage where the raw scaling on compute happens. It involves training the model on a vast amount of data to develop a general understanding of language and common sense.\n\n2. **General Common Sense**: Without substantial pre-training, the model lacks the baseline common sense necessary for effective reasoning. This foundational knowledge is critical because it equips the model with a broad understanding of language and concepts.\n\n3. **Importance in Combination with Post-Training**: While post-training (which includes supervised fine-tuning and reinforcement learning from human feedback, or RLHF) is essential for refining and controlling the mode

In [ ]:
perform_rag("What advantages does Perplexity have over other AI companies?")

"Perplexity differentiates itself from other AI companies by focusing on a few unique aspects:\n\n1. **Answer-Centric Approach**: Unlike traditional search engines that display a list of URLs, Perplexity aims to provide direct, Wikipedia-like responses to queries. This method prioritizes giving users direct answers and relevant information over sending them to another webpage. This shifts the UI focus from a list of links to summarized answers, aiming to provide a more streamlined and valuable user experience.\n\n2. **Factual Grounding (RAG - Retrieval-Augmented Generation)**: Perplexity ensures their answers are factually grounded by only generating responses based on documents retrieved from the internet. This principle aims to reduce hallucinations by sticking closely to the retrieved content, enhancing the trustworthiness and accuracy of the information provided.\n\n3. **Knowledge-Centric Mission**: The company’s mission goes beyond search and aims to make people smarter by helping

# RAG over a PDF

In [ ]:
loader = PyPDFLoader("/content/Harry Potter and the Sorcerers Stone.pdf") # Insert the path to a PDF here
data = loader.load()

print(data)

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=100,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
    )

texts = text_splitter.split_documents(data)

# Insert all the chunks from the PDF into Pinecone
vectorstore_from_texts = PineconeVectorStore.from_texts([f"Source: {t.metadata['source']}, Title: {t.metadata['title']} \n\nContent: {t.page_content}" for t in texts], embeddings, index_name=index_name, namespace=namespace)

# After this, all the code is the same from the Perform RAG section of this notebook
# Since the data from the PDF is now stored in Pinecone, you can perform RAG over it the same way as the YouTube video